<a href="https://colab.research.google.com/github/estersantos2004/ester/blob/main/Conhe%C3%A7a_o_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import openai
from google.colab import userdata

# Use Colab Secrets to store your API key
chave_api = userdata.get('OPENAI_API_KEY')

client = openai.OpenAI(api_key=chave_api)

def enviar_mensagem(mensagem):
    resposta = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": mensagem}
        ],
    )
    return resposta.choices[0].message


while True:
  texto = input("escreva aqui sua mensagem:")

  if texto == "sair":
    break
  else:
       resposta = enviar_mensagem(texto)
       print("chatbot:", resposta.content)

Para usar a API do Gemini, você precisará de uma chave de API. Se você ainda não tiver uma, crie uma no Google AI Studio.
No Colab, adicione a chave ao gerenciador de segredos (Secrets Manager) clicando no ícone "🔑" no painel esquerdo. Dê a ela o nome `GOOGLE_API_KEY`. Em seguida, passe a chave para o SDK:

In [ ]:
# Instale a biblioteca Python do SDK do Google para a API Gemini
!pip install -q -U google-generativeai

# Importe a biblioteca Python do SDK
import google.generativeai as genai
# Usado para armazenar sua chave de API de forma segura
from google.colab import userdata

# Obtenha a chave de API do Secrets Manager do Colab
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

# Configure a API com sua chave
genai.configure(api_key=GOOGLE_API_KEY)

Antes de fazer qualquer chamada de API, você precisa inicializar o modelo generativo.

In [ ]:
# Inicialize o modelo Gemini. Você pode escolher um modelo diferente se necessário.
# Veja os modelos disponíveis com `for m in genai.list_models(): print(m.name)`

# Listando modelos disponíveis para verificar qual usar
print("Modelos disponíveis:")
for m in genai.list_models():
  print(f"- {m.name}")

# Substitua 'gemini-pro' por um modelo válido da lista acima, se necessário
gemini_model = genai.GenerativeModel('models/gemini-pro-latest')

Agora você pode fazer chamadas de API. Por exemplo, para gerar um texto com base em um prompt:

In [ ]:
prompt = "Escreva uma breve história sobre um robô amigável que ajuda as pessoas."
response = gemini_model.generate_content(prompt)

# Imprima o texto gerado pela resposta
print(response.text)

# Task
Completar o código do chatbot para que ele possa processar pedidos com base na tabela de preços fornecida.

## Completar o treinamento do chatbot

### Subtask:
Adicionar mais frases de treinamento relacionadas a pedidos, como perguntar sobre itens, códigos e finalizar o pedido.


**Reasoning**:
The user wants to add more training phrases for a chatbot related to placing orders. This involves creating a list of phrases and associating them with relevant intents. I will create a list of training phrases related to ordering and then integrate them into a structure that can be used for training, assuming a dictionary or similar structure is used to hold intents and their corresponding phrases. Since the previous code blocks don't define the training data structure, I will create a sample structure to demonstrate how the new phrases would be added.



In [ ]:
# Assuming a dictionary structure for intents and training phrases
training_data = {
    "saudacao": [
        "Olá",
        "Oi",
        "Bom dia",
        "Boa tarde",
        "Boa noite"
    ],
    "despedida": [
        "Tchau",
        "Adeus",
        "Até mais"
    ],
    "agradecimento": [
        "Obrigado",
        "Obrigada",
        "Valeu"
    ],
    # Adding new training phrases for ordering
    "pedido": [
        "Quero fazer um pedido",
        "Gostaria de pedir algo",
        "Posso fazer um pedido?",
        "Iniciar pedido"
    ],
    "adicionar_item": [
        "Adicionar item",
        "Quero adicionar",
        "Incluir",
        "Adicionar o produto X",
        "Quero o item com código Y",
        "Adicionar código Z",
        "Mais um do item A"
    ],
    "finalizar_pedido": [
        "Finalizar pedido",
        "Concluir pedido",
        "Terminar meu pedido",
        "Pode fechar a conta",
        "Encerrar pedido"
    ]
}

# Display the updated training data (optional, for verification)
import json
print(json.dumps(training_data, indent=2))

## Implementar a lógica de processamento de pedidos

### Subtask:
Adicionar código para identificar quando o usuário está fazendo um pedido, extrair as informações relevantes (como o código do item e a quantidade) e consultar a `tabela_de_precos`.


**Reasoning**:
Define the price list and create a function to process user messages for order intent and item extraction, referencing the training data for keywords.



In [ ]:
# 1. Define the price list
tabela_de_precos = {
    "item X": 10.50,
    "item Y": 25.00,
    "item Z": 5.75,
    "item A": 15.00,
    "código X": 10.50,
    "código Y": 25.00,
    "código Z": 5.75,
    "código A": 15.00,
}

# 2. Create a function to process user input for orders
def processar_pedido(mensagem, tabela_de_precos, training_data):
    mensagem_lower = mensagem.lower()

    # 3. Implement logic to detect order intent and extract information
    for intent, phrases in training_data.items():
        if intent in ["pedido", "adicionar_item", "finalizar_pedido"]:
            for phrase in phrases:
                if phrase.lower() in mensagem_lower:
                    # Simple extraction logic (can be improved with regex or more advanced methods)
                    for item_code in tabela_de_precos.keys():
                        if item_code.lower() in mensagem_lower:
                            # Assuming quantity is 1 if not specified, or try to extract a number
                            # This is a basic example and needs refinement for real-world use
                            quantity = 1
                            import re
                            match = re.search(r'(\d+)\s+' + re.escape(item_code.lower()), mensagem_lower)
                            if match:
                                quantity = int(match.group(1))

                            # 5. Consult the price list
                            price = tabela_de_precos[item_code]

                            # 6. Return extracted information
                            return {"intent": intent, "item": item_code, "quantity": quantity, "price": price}

    # Return indication if no relevant item was found
    return {"intent": "none"}

# Example usage (for testing the function)
mensagem_teste_1 = "Quero adicionar 2 do item X"
resultado_1 = processar_pedido(mensagem_teste_1, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_1}', Resultado: {resultado_1}")

mensagem_teste_2 = "Finalizar meu pedido"
resultado_2 = processar_pedido(mensagem_teste_2, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_2}', Resultado: {resultado_2}")

mensagem_teste_3 = "Olá, tudo bem?"
resultado_3 = processar_pedido(mensagem_teste_3, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_3}', Resultado: {resultado_3}")

mensagem_teste_4 = "Quero o item com código Y"
resultado_4 = processar_pedido(mensagem_teste_4, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_4}', Resultado: {resultado_4}")


**Reasoning**:
The function successfully extracts item information but the quantity extraction is not working as expected. The next step is to refine the quantity extraction logic using regular expressions within the `processar_pedido` function.



In [ ]:
# Refine the quantity extraction logic within the processar_pedido function

# Re-defining the function with improved quantity extraction
def processar_pedido(mensagem, tabela_de_precos, training_data):
    mensagem_lower = mensagem.lower()

    for intent, phrases in training_data.items():
        if intent in ["pedido", "adicionar_item", "finalizar_pedido"]:
            for phrase in phrases:
                if phrase.lower() in mensagem_lower:
                    for item_code in tabela_de_precos.keys():
                        # Improved extraction logic using regex to find quantity before the item name/code
                        # This regex looks for a number followed by optional space(s) before the item code
                        import re
                        match = re.search(r'(\d+)\s*' + re.escape(item_code.lower()), mensagem_lower)

                        quantity = 1 # Default quantity is 1
                        if match:
                            quantity = int(match.group(1))

                        if item_code.lower() in mensagem_lower:
                             price = tabela_de_precos[item_code]
                             return {"intent": intent, "item": item_code, "quantity": quantity, "price": price}


    return {"intent": "none"}

# Example usage with the refined function
mensagem_teste_1 = "Quero adicionar 2 do item X"
resultado_1 = processar_pedido(mensagem_teste_1, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_1}', Resultado: {resultado_1}")

mensagem_teste_2 = "Finalizar meu pedido"
resultado_2 = processar_pedido(mensagem_teste_2, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_2}', Resultado: {resultado_2}")

mensagem_teste_3 = "Olá, tudo bem?"
resultado_3 = processar_pedido(mensagem_teste_3, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_3}', Resultado: {resultado_3}")

mensagem_teste_4 = "Quero o item com código Y"
resultado_4 = processar_pedido(mensagem_teste_4, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_4}', Resultado: {resultado_4}")

mensagem_teste_5 = "Adicionar 5 do item A"
resultado_5 = processar_pedido(mensagem_teste_5, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_5}', Resultado: {resultado_5}")

**Reasoning**:
The quantity extraction still doesn't work correctly and the detection of "Finalizar meu pedido" as an intent is also failing. I need to further refine the regex for quantity extraction and adjust the intent matching logic to be more robust.



In [ ]:
# Further refine quantity extraction and intent matching
import re

def processar_pedido(mensagem, tabela_de_precos, training_data):
    mensagem_lower = mensagem.lower()

    # Check for specific intents first
    for intent, phrases in training_data.items():
        if intent in ["pedido", "finalizar_pedido"]:
            for phrase in phrases:
                if phrase.lower() in mensagem_lower:
                    return {"intent": intent}

    # If not a simple intent, try to extract item and quantity
    for item_code in tabela_de_precos.keys():
        # Look for item code and optional quantity before it
        match = re.search(r'(\d+)?\s*' + re.escape(item_code.lower()), mensagem_lower)
        if match:
            quantity_str = match.group(1)
            quantity = int(quantity_str) if quantity_str else 1
            price = tabela_de_precos[item_code]
            return {"intent": "adicionar_item", "item": item_code, "quantity": quantity, "price": price}

    # Return indication if no relevant item or specific intent was found
    return {"intent": "none"}

# Example usage with the refined function
mensagem_teste_1 = "Quero adicionar 2 do item X"
resultado_1 = processar_pedido(mensagem_teste_1, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_1}', Resultado: {resultado_1}")

mensagem_teste_2 = "Finalizar meu pedido"
resultado_2 = processar_pedido(mensagem_teste_2, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_2}', Resultado: {resultado_2}")

mensagem_teste_3 = "Olá, tudo bem?"
resultado_3 = processar_pedido(mensagem_teste_3, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_3}', Resultado: {resultado_3}")

mensagem_teste_4 = "Quero o item com código Y"
resultado_4 = processar_pedido(mensagem_teste_4, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_4}', Resultado: {resultado_4}")

mensagem_teste_5 = "Adicionar 5 do item A"
resultado_5 = processar_pedido(mensagem_teste_5, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_5}', Resultado: {resultado_5}")

mensagem_teste_6 = "item Z"
resultado_6 = processar_pedido(mensagem_teste_6, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_6}', Resultado: {resultado_6}")

**Reasoning**:
The quantity extraction is still defaulting to 1, and the "Finalizar meu pedido" intent is not being correctly identified. The regex needs to be adjusted to correctly capture the quantity before the item name/code, and the intent matching should prioritize exact matches for specific phrases like "finalizar pedido".



In [ ]:
# Further refine quantity extraction and intent matching

def processar_pedido(mensagem, tabela_de_precos, training_data):
    mensagem_lower = mensagem.lower()

    # Prioritize exact matches for specific intents like 'finalizar_pedido'
    for phrase in training_data.get("finalizar_pedido", []):
        if phrase.lower() in mensagem_lower:
            return {"intent": "finalizar_pedido"}

    for phrase in training_data.get("pedido", []):
         if phrase.lower() in mensagem_lower:
            return {"intent": "pedido"}


    # Then, try to extract item and quantity for 'adicionar_item'
    for item_code in tabela_de_precos.keys():
        # Look for item code and optional quantity before it
        # Updated regex to correctly capture the quantity
        match = re.search(r'(\d+)?\s*' + re.escape(item_code.lower()), mensagem_lower)
        if match:
            quantity_str = match.group(1)
            quantity = int(quantity_str) if quantity_str else 1
            price = tabela_de_precos[item_code]
            return {"intent": "adicionar_item", "item": item_code, "quantity": quantity, "price": price}

    # Return indication if no relevant item or specific intent was found
    return {"intent": "none"}

# Example usage with the refined function
mensagem_teste_1 = "Quero adicionar 2 do item X"
resultado_1 = processar_pedido(mensagem_teste_1, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_1}', Resultado: {resultado_1}")

mensagem_teste_2 = "Finalizar meu pedido"
resultado_2 = processar_pedido(mensagem_teste_2, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_2}', Resultado: {resultado_2}")

mensagem_teste_3 = "Olá, tudo bem?"
resultado_3 = processar_pedido(mensagem_teste_3, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_3}', Resultado: {resultado_3}")

mensagem_teste_4 = "Quero o item com código Y"
resultado_4 = processar_pedido(mensagem_teste_4, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_4}', Resultado: {resultado_4}")

mensagem_teste_5 = "Adicionar 5 do item A"
resultado_5 = processar_pedido(mensagem_teste_5, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_5}', Resultado: {resultado_5}")

mensagem_teste_6 = "item Z"
resultado_6 = processar_pedido(mensagem_teste_6, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_6}', Resultado: {resultado_6}")

mensagem_teste_7 = "Iniciar pedido"
resultado_7 = processar_pedido(mensagem_teste_7, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_7}', Resultado: {resultado_7}")

**Reasoning**:
The quantity extraction is still consistently failing to capture the number, and the "Finalizar meu pedido" intent is not being recognized. The regex for quantity needs to be re-evaluated, and the intent matching for "finalizar_pedido" should be made more robust, perhaps by checking for the exact phrase after lowercasing and stripping leading/trailing whitespace.



In [ ]:
# Further refine quantity extraction and intent matching

def processar_pedido(mensagem, tabela_de_precos, training_data):
    mensagem_lower = mensagem.lower().strip() # Strip whitespace

    # Prioritize exact matches for specific intents like 'finalizar_pedido'
    for phrase in training_data.get("finalizar_pedido", []):
        if phrase.lower() == mensagem_lower: # Check for exact match
            return {"intent": "finalizar_pedido"}

    for phrase in training_data.get("pedido", []):
         if phrase.lower() == mensagem_lower: # Check for exact match
            return {"intent": "pedido"}


    # Then, try to extract item and quantity for 'adicionar_item'
    for item_code in tabela_de_precos.keys():
        # Look for item code and optional quantity before it
        # Updated regex to correctly capture the quantity, allowing for optional space
        match = re.search(r'(\d+)\s+' + re.escape(item_code.lower()), mensagem_lower)
        quantity = 1 # Default quantity is 1
        if match:
            quantity = int(match.group(1))
            price = tabela_de_precos[item_code]
            return {"intent": "adicionar_item", "item": item_code, "quantity": quantity, "price": price}

        # Also check for item code without a preceding quantity (defaults to quantity 1)
        if item_code.lower() in mensagem_lower and not match:
             price = tabela_de_precos[item_code]
             return {"intent": "adicionar_item", "item": item_code, "quantity": 1, "price": price}


    # Return indication if no relevant item or specific intent was found
    return {"intent": "none"}

# Example usage with the refined function
mensagem_teste_1 = "Quero adicionar 2 do item X"
resultado_1 = processar_pedido(mensagem_teste_1, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_1}', Resultado: {resultado_1}")

mensagem_teste_2 = "Finalizar meu pedido"
resultado_2 = processar_pedido(mensagem_teste_2, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_2}', Resultado: {resultado_2}")

mensagem_teste_3 = "Olá, tudo bem?"
resultado_3 = processar_pedido(mensagem_teste_3, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_3}', Resultado: {resultado_3}")

mensagem_teste_4 = "Quero o item com código Y"
resultado_4 = processar_pedido(mensagem_teste_4, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_4}', Resultado: {resultado_4}")

mensagem_teste_5 = "Adicionar 5 do item A"
resultado_5 = processar_pedido(mensagem_teste_5, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_5}', Resultado: {resultado_5}")

mensagem_teste_6 = "item Z"
resultado_6 = processar_pedido(mensagem_teste_6, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_6}', Resultado: {resultado_6}")

mensagem_teste_7 = "Iniciar pedido"
resultado_7 = processar_pedido(mensagem_teste_7, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_7}', Resultado: {resultado_7}")

mensagem_teste_8 = "Posso fazer um pedido?"
resultado_8 = processar_pedido(mensagem_teste_8, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_8}', Resultado: {resultado_8}")


**Reasoning**:
The quantity extraction still fails to capture the number. The issue might be with how the regex is constructed or how the match is being processed. I will modify the regex to be more explicit about capturing the digit group before the item code and adjust the logic to ensure the item code is present in the message before attempting to extract quantity. I also need to fix the "Finalizar meu pedido" intent detection, which is still failing for the example "Finalizar meu pedido". The exact match logic implemented in the previous step seems to be the issue. I will revert to checking if the phrase is *in* the lowercased message for intent detection.



In [ ]:
# Further refine quantity extraction and intent matching

def processar_pedido(mensagem, tabela_de_precos, training_data):
    mensagem_lower = mensagem.lower()

    # Prioritize checking for specific intents
    for intent, phrases in training_data.items():
        if intent in ["pedido", "finalizar_pedido"]:
            for phrase in phrases:
                if phrase.lower() in mensagem_lower: # Check if phrase is in the message
                    return {"intent": intent}

    # Then, try to extract item and quantity for 'adicionar_item'
    for item_code in tabela_de_precos.keys():
        item_code_lower = item_code.lower()
        if item_code_lower in mensagem_lower:
            # Look for an optional quantity before the item code
            # Updated regex to explicitly capture digits followed by space or end of string before the item code
            match = re.search(r'(\d+)\s+' + re.escape(item_code_lower), mensagem_lower)
            quantity = 1 # Default quantity is 1
            if match:
                quantity = int(match.group(1))

            price = tabela_de_precos[item_code]
            return {"intent": "adicionar_item", "item": item_code, "quantity": quantity, "price": price}


    # Return indication if no relevant item or specific intent was found
    return {"intent": "none"}

# Example usage with the refined function
mensagem_teste_1 = "Quero adicionar 2 do item X"
resultado_1 = processar_pedido(mensagem_teste_1, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_1}', Resultado: {resultado_1}")

mensagem_teste_2 = "Finalizar meu pedido"
resultado_2 = processar_pedido(mensagem_teste_2, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_2}', Resultado: {resultado_2}")

mensagem_teste_3 = "Olá, tudo bem?"
resultado_3 = processar_pedido(mensagem_teste_3, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_3}', Resultado: {resultado_3}")

mensagem_teste_4 = "Quero o item com código Y"
resultado_4 = processar_pedido(mensagem_teste_4, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_4}', Resultado: {resultado_4}")

mensagem_teste_5 = "Adicionar 5 do item A"
resultado_5 = processar_pedido(mensagem_teste_5, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_5}', Resultado: {resultado_5}")

mensagem_teste_6 = "item Z"
resultado_6 = processar_pedido(mensagem_teste_6, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_6}', Resultado: {resultado_6}")

mensagem_teste_7 = "Iniciar pedido"
resultado_7 = processar_pedido(mensagem_teste_7, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_7}', Resultado: {resultado_7}")

mensagem_teste_8 = "Posso fazer um pedido?"
resultado_8 = processar_pedido(mensagem_teste_8, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_8}', Resultado: {resultado_8}")

**Reasoning**:
The quantity extraction is still not working, and the "Finalizar meu pedido" intent is not being detected. I will try a different approach for quantity extraction, looking for digits directly preceding the item name/code. I will also ensure the intent matching for "finalizar_pedido" correctly identifies the phrase within the message.



In [ ]:
# Final attempt to refine quantity extraction and intent matching

def processar_pedido(mensagem, tabela_de_precos, training_data):
    mensagem_lower = mensagem.lower()

    # Prioritize checking for specific intents
    for intent, phrases in training_data.items():
        if intent in ["pedido", "finalizar_pedido"]:
            for phrase in phrases:
                if phrase.lower() in mensagem_lower: # Check if phrase is in the message
                    return {"intent": intent}

    # Then, try to extract item and quantity for 'adicionar_item'
    for item_code in tabela_de_precos.keys():
        item_code_lower = item_code.lower()
        if item_code_lower in mensagem_lower:
            # Look for digits immediately preceding the item code, with optional space
            match = re.search(r'(\d+)\s*' + re.escape(item_code_lower), mensagem_lower)
            quantity = 1 # Default quantity is 1
            if match:
                quantity = int(match.group(1))

            price = tabela_de_precos[item_code]
            return {"intent": "adicionar_item", "item": item_code, "quantity": quantity, "price": price}

    # Return indication if no relevant item or specific intent was found
    return {"intent": "none"}

# Example usage with the refined function
mensagem_teste_1 = "Quero adicionar 2 do item X"
resultado_1 = processar_pedido(mensagem_teste_1, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_1}', Resultado: {resultado_1}")

mensagem_teste_2 = "Finalizar meu pedido"
resultado_2 = processar_pedido(mensagem_teste_2, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_2}', Resultado: {resultado_2}")

mensagem_teste_3 = "Olá, tudo bem?"
resultado_3 = processar_pedido(mensagem_teste_3, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_3}', Resultado: {resultado_3}")

mensagem_teste_4 = "Quero o item com código Y"
resultado_4 = processar_pedido(mensagem_teste_4, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_4}', Resultado: {resultado_4}")

mensagem_teste_5 = "Adicionar 5 do item A"
resultado_5 = processar_pedido(mensagem_teste_5, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_5}', Resultado: {resultado_5}")

mensagem_teste_6 = "item Z"
resultado_6 = processar_pedido(mensagem_teste_6, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_6}', Resultado: {resultado_6}")

mensagem_teste_7 = "Iniciar pedido"
resultado_7 = processar_pedido(mensagem_teste_7, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_7}', Resultado: {resultado_7}")

mensagem_teste_8 = "Posso fazer um pedido?"
resultado_8 = processar_pedido(mensagem_teste_8, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_8}', Resultado: {resultado_8}")


## Implementar a lógica de processamento de pedidos

### Subtask:
Implementar a lógica de processamento de pedidos, incluindo a identificação da intenção do usuário, extração de informações relevantes (item, quantidade) e consulta à tabela de preços.


**Reasoning**:
Refine the `processar_pedido` function to improve intent detection and quantity extraction, then test with various messages.



In [ ]:
import re

def processar_pedido(mensagem, tabela_de_precos, training_data):
    mensagem_lower = mensagem.lower()

    # Check for specific intents first with a more robust approach
    # Use regex to find exact phrases for "finalizar_pedido"
    for phrase in training_data.get("finalizar_pedido", []):
        if re.search(r'\b' + re.escape(phrase.lower()) + r'\b', mensagem_lower):
            return {"intent": "finalizar_pedido"}

    # Use regex to find exact phrases for "pedido"
    for phrase in training_data.get("pedido", []):
         if re.search(r'\b' + re.escape(phrase.lower()) + r'\b', mensagem_lower):
            return {"intent": "pedido"}


    # Then, try to extract item and quantity for 'adicionar_item'
    for item_code in tabela_de_precos.keys():
        item_code_lower = item_code.lower()
        # Look for an optional quantity before the item code, allowing for different phrasing
        # This regex looks for a number followed by optional words like "do", "de", "dos", "das" and then the item code
        match = re.search(r'(\d+)\s+(?:do|de|dos|das)?\s*' + re.escape(item_code_lower), mensagem_lower)
        quantity = 1 # Default quantity is 1
        if match:
            quantity = int(match.group(1))
            price = tabela_de_precos[item_code]
            return {"intent": "adicionar_item", "item": item_code, "quantity": quantity, "price": price}

        # Also check for item code without a preceding quantity (defaults to quantity 1)
        if re.search(r'\b' + re.escape(item_code_lower) + r'\b', mensagem_lower) and not match:
             price = tabela_de_precos[item_code]
             return {"intent": "adicionar_item", "item": item_code, "quantity": 1, "price": price}

    # Return indication if no relevant item or specific intent was found
    return {"intent": "none"}

# Test the refined function with various messages
mensagem_teste_1 = "Quero adicionar 2 do item X"
resultado_1 = processar_pedido(mensagem_teste_1, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_1}', Resultado: {resultado_1}")

mensagem_teste_2 = "Finalizar meu pedido"
resultado_2 = processar_pedido(mensagem_teste_2, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_2}', Resultado: {resultado_2}")

mensagem_teste_3 = "Olá, tudo bem?"
resultado_3 = processar_pedido(mensagem_teste_3, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_3}', Resultado: {resultado_3}")

mensagem_teste_4 = "Quero o item com código Y"
resultado_4 = processar_pedido(mensagem_teste_4, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_4}', Resultado: {resultado_4}")

mensagem_teste_5 = "Adicionar 5 do item A"
resultado_5 = processar_pedido(mensagem_teste_5, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_5}', Resultado: {resultado_5}")

mensagem_teste_6 = "item Z"
resultado_6 = processar_pedido(mensagem_teste_6, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_6}', Resultado: {resultado_6}")

mensagem_teste_7 = "Iniciar pedido"
resultado_7 = processar_pedido(mensagem_teste_7, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_7}', Resultado: {resultado_7}")

mensagem_teste_8 = "Posso fazer um pedido?"
resultado_8 = processar_pedido(mensagem_teste_8, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_8}', Resultado: {resultado_8}")

# Additional test cases for better coverage
mensagem_teste_9 = "adicionar 3 itens do código X"
resultado_9 = processar_pedido(mensagem_teste_9, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_9}', Resultado: {resultado_9}")

mensagem_teste_10 = "quatro do item Y por favor"
resultado_10 = processar_pedido(mensagem_teste_10, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_10}', Resultado: {resultado_10}")

mensagem_teste_11 = "Concluir pedido"
resultado_11 = processar_pedido(mensagem_teste_11, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_11}', Resultado: {resultado_11}")

mensagem_teste_12 = "gostaria de adicionar item A"
resultado_12 = processar_pedido(mensagem_teste_12, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_12}', Resultado: {resultado_12}")

**Reasoning**:
Further refine the intent detection and quantity extraction logic based on the previous test results, then retest with all messages.



In [ ]:
import re

def processar_pedido(mensagem, tabela_de_precos, training_data):
    mensagem_lower = mensagem.lower()

    # Prioritize checking for specific intents with improved regex
    for phrase in training_data.get("finalizar_pedido", []):
        # Use word boundaries to ensure exact phrase match
        if re.search(r'\b' + re.escape(phrase.lower()) + r'\b', mensagem_lower):
            return {"intent": "finalizar_pedido"}

    for phrase in training_data.get("pedido", []):
        # Use word boundaries for exact phrase match
        if re.search(r'\b' + re.escape(phrase.lower()) + r'\b', mensagem_lower):
            return {"intent": "pedido"}

    # Then, try to extract item and quantity for 'adicionar_item'
    for item_code in tabela_de_precos.keys():
        item_code_lower = item_code.lower()
        # Look for an optional quantity before the item code, allowing for different phrasing and optional spaces.
        # This regex looks for digits followed by optional words like "do", "de", "dos", "das", optional spaces, and then the item code.
        match = re.search(r'(\d+)\s*(?:do|de|dos|das)?\s*' + re.escape(item_code_lower), mensagem_lower)
        quantity = 1 # Default quantity is 1
        if match:
            quantity = int(match.group(1))
            price = tabela_de_precos[item_code]
            return {"intent": "adicionar_item", "item": item_code, "quantity": quantity, "price": price}

        # Also check for item code without a preceding quantity (defaults to quantity 1)
        # Ensure it's a whole word match for the item code
        if re.search(r'\b' + re.escape(item_code_lower) + r'\b', mensagem_lower) and not match:
             price = tabela_de_precos[item_code]
             return {"intent": "adicionar_item", "item": item_code, "quantity": 1, "price": price}


    # Return indication if no relevant item or specific intent was found
    return {"intent": "none"}

# Test the refined function with various messages
mensagem_teste_1 = "Quero adicionar 2 do item X"
resultado_1 = processar_pedido(mensagem_teste_1, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_1}', Resultado: {resultado_1}")

mensagem_teste_2 = "Finalizar meu pedido"
resultado_2 = processar_pedido(mensagem_teste_2, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_2}', Resultado: {resultado_2}")

mensagem_teste_3 = "Olá, tudo bem?"
resultado_3 = processar_pedido(mensagem_teste_3, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_3}', Resultado: {resultado_3}")

mensagem_teste_4 = "Quero o item com código Y"
resultado_4 = processar_pedido(mensagem_teste_4, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_4}', Resultado: {resultado_4}")

mensagem_teste_5 = "Adicionar 5 do item A"
resultado_5 = processar_pedido(mensagem_teste_5, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_5}', Resultado: {resultado_5}")

mensagem_teste_6 = "item Z"
resultado_6 = processar_pedido(mensagem_teste_6, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_6}', Resultado: {resultado_6}")

mensagem_teste_7 = "Iniciar pedido"
resultado_7 = processar_pedido(mensagem_teste_7, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_7}', Resultado: {resultado_7}")

mensagem_teste_8 = "Posso fazer um pedido?"
resultado_8 = processar_pedido(mensagem_teste_8, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_8}', Resultado: {resultado_8}")

mensagem_teste_9 = "adicionar 3 itens do código X"
resultado_9 = processar_pedido(mensagem_teste_9, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_9}', Resultado: {resultado_9}")

mensagem_teste_10 = "quatro do item Y por favor"
resultado_10 = processar_pedido(mensagem_teste_10, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_10}', Resultado: {resultado_10}")

mensagem_teste_11 = "Concluir pedido"
resultado_11 = processar_pedido(mensagem_teste_11, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_11}', Resultado: {resultado_11}")

mensagem_teste_12 = "gostaria de adicionar item A"
resultado_12 = processar_pedido(mensagem_teste_12, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_12}', Resultado: {resultado_12}")

**Reasoning**:
Based on the previous test results, the intent detection for "Finalizar meu pedido" and "Posso fazer um pedido?" is not working correctly. Also, the quantity extraction for some cases like "adicionar 3 itens do código X" and "quatro do item Y por favor" is failing. I need to further refine the regex for both intent detection and quantity extraction to address these issues.



In [ ]:
import re

def processar_pedido(mensagem, tabela_de_precos, training_data):
    mensagem_lower = mensagem.lower()

    # Prioritize checking for specific intents with improved regex
    for phrase in training_data.get("finalizar_pedido", []):
        # Use word boundaries and allow for some variations in phrasing
        if re.search(r'.*' + re.escape(phrase.lower()) + r'.*', mensagem_lower):
            return {"intent": "finalizar_pedido"}

    for phrase in training_data.get("pedido", []):
        # Use word boundaries and allow for some variations
        if re.search(r'.*' + re.escape(phrase.lower()) + r'.*', mensagem_lower):
            return {"intent": "pedido"}

    # Then, try to extract item and quantity for 'adicionar_item'
    for item_code in tabela_de_precos.keys():
        item_code_lower = item_code.lower()
        # Look for an optional quantity before the item code, allowing for different phrasing and optional spaces.
        # This regex looks for digits followed by optional words like "do", "de", "dos", "das", optional spaces, and then the item code.
        # It also handles cases like "3 itens do código X" by looking for "itens" after the quantity.
        match = re.search(r'(\d+)\s*(?:itens?\s*)?(?:do|de|dos|das)?\s*' + re.escape(item_code_lower), mensagem_lower)
        quantity = 1 # Default quantity is 1
        if match:
            quantity = int(match.group(1))
            price = tabela_de_precos[item_code]
            return {"intent": "adicionar_item", "item": item_code, "quantity": quantity, "price": price}

        # Also check for item code without a preceding quantity (defaults to quantity 1)
        # Ensure it's a whole word match for the item code
        if re.search(r'\b' + re.escape(item_code_lower) + r'\b', mensagem_lower) and not match:
             price = tabela_de_precos[item_code]
             return {"intent": "adicionar_item", "item": item_code, "quantity": 1, "price": price}


    # Return indication if no relevant item or specific intent was found
    return {"intent": "none"}

# Test the refined function with various messages
mensagem_teste_1 = "Quero adicionar 2 do item X"
resultado_1 = processar_pedido(mensagem_teste_1, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_1}', Resultado: {resultado_1}")

mensagem_teste_2 = "Finalizar meu pedido"
resultado_2 = processar_pedido(mensagem_teste_2, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_2}', Resultado: {resultado_2}")

mensagem_teste_3 = "Olá, tudo bem?"
resultado_3 = processar_pedido(mensagem_teste_3, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_3}', Resultado: {resultado_3}")

mensagem_teste_4 = "Quero o item com código Y"
resultado_4 = processar_pedido(mensagem_teste_4, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_4}', Resultado: {resultado_4}")

mensagem_teste_5 = "Adicionar 5 do item A"
resultado_5 = processar_pedido(mensagem_teste_5, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_5}', Resultado: {resultado_5}")

mensagem_teste_6 = "item Z"
resultado_6 = processar_pedido(mensagem_teste_6, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_6}', Resultado: {resultado_6}")

mensagem_teste_7 = "Iniciar pedido"
resultado_7 = processar_pedido(mensagem_teste_7, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_7}', Resultado: {resultado_7}")

mensagem_teste_8 = "Posso fazer um pedido?"
resultado_8 = processar_pedido(mensagem_teste_8, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_8}', Resultado: {resultado_8}")

mensagem_teste_9 = "adicionar 3 itens do código X"
resultado_9 = processar_pedido(mensagem_teste_9, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_9}', Resultado: {resultado_9}")

mensagem_teste_10 = "quatro do item Y por favor"
resultado_10 = processar_pedido(mensagem_teste_10, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_10}', Resultado: {resultado_10}")

mensagem_teste_11 = "Concluir pedido"
resultado_11 = processar_pedido(mensagem_teste_11, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_11}', Resultado: {resultado_11}")

mensagem_teste_12 = "gostaria de adicionar item A"
resultado_12 = processar_pedido(mensagem_teste_12, tabela_de_precos, training_data)
print(f"Mensagem: '{mensagem_teste_12}', Resultado: {resultado_12}")

## Calcular o total do pedido

### Subtask:
Adicionar funcionalidade para manter um registro dos itens no pedido atual e calcular o custo total.


**Reasoning**:
Initialize an empty list to store the order items and modify the code to process the results of the `processar_pedido` function to add items to the order or calculate the total for finalizing the order.



In [ ]:
# Initialize an empty list to store the current order
current_order = []

# Modify the main loop to process user input and manage the order
while True:
    texto = input("escreva aqui sua mensagem:")

    if texto == "sair":
        break
    else:
        # Process the user's message to identify intent and extract information
        resultado = processar_pedido(texto, tabela_de_precos, training_data)

        # Handle the identified intent
        if resultado["intent"] == "adicionar_item":
            item = resultado["item"]
            quantity = resultado["quantity"]
            price = resultado["price"]
            subtotal = quantity * price
            current_order.append({"item": item, "quantity": quantity, "subtotal": subtotal})
            print(f"Adicionado {quantity} de {item} ao pedido. Subtotal para este item: R${subtotal:.2f}")
        elif resultado["intent"] == "finalizar_pedido":
            if not current_order:
                print("Seu pedido está vazio.")
            else:
                total_cost = sum(item["subtotal"] for item in current_order)
                print("Seu pedido:")
                for item in current_order:
                    print(f"- {item['quantity']}x {item['item']}: R${item['subtotal']:.2f}")
                print(f"Total do pedido: R${total_cost:.2f}")
                # Clear the order after finalizing
                current_order = []
        elif resultado["intent"] == "pedido":
            print("Ok, o que você gostaria de pedir?")
        else:
            # If no specific order intent is detected, you could potentially
            # pass the message to the general chatbot model here.
            # For this subtask, we'll just indicate that the input wasn't understood
            # in the context of an order.
            print("Desculpe, não entendi seu pedido. Você pode adicionar itens ou finalizar o pedido.")


escreva aqui sua mensagem:em que ano einstein publicou a teoria da relatividade?
Desculpe, não entendi seu pedido. Você pode adicionar itens ou finalizar o pedido.
escreva aqui sua mensagem:oii
Desculpe, não entendi seu pedido. Você pode adicionar itens ou finalizar o pedido.
escreva aqui sua mensagem:quero gerar meu pedido
Desculpe, não entendi seu pedido. Você pode adicionar itens ou finalizar o pedido.
escreva aqui sua mensagem:oi
Desculpe, não entendi seu pedido. Você pode adicionar itens ou finalizar o pedido.
